In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("/home/ace/vertika/test/train.csv")

In [3]:
print(data.head())

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [4]:
train_x = data.values[:,1:]
train_y = data.values[:,0]
print(train_x)
print(train_y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 0 1 ... 7 6 9]


In [5]:
split_size = int(train_x.shape[0]*0.7)
print(split_size)

29399


In [6]:
train_x, val_x = train_x[:split_size, : ], train_x[split_size: , :]
print(train_x.shape)
print(len(train_x))
print(val_x.shape)

train_y, val_y = train_y[:split_size], train_y[split_size:]
print(train_y.shape)
print(val_y.shape)

(29399, 784)
29399
(12601, 784)
(29399,)
(12601,)


In [7]:
test_x = pd.read_csv( "/home/ace/vertika/test/test.csv"  ).values

In [8]:
test_x.shape

(28000, 784)

In [9]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score

seed = 128
rng = np.random.RandomState(seed)


# set remaining variables
epochs = 2
batch_size = 128
learning_rate = 0.01

In [10]:

# number of neurons in each layer
input_num_units = 28*28
hidden_num_units = 50
output_num_units = 10

# define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])



weights = {
    'hidden': tf.Variable(tf.random_normal([input_num_units, hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.random_normal([hidden_num_units, output_num_units], seed=seed))
}

biases = {
    'hidden': tf.Variable(tf.random_normal([hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.random_normal([output_num_units], seed=seed))
}

weights


{'hidden': <tf.Variable 'Variable:0' shape=(784, 50) dtype=float32_ref>,
 'output': <tf.Variable 'Variable_1:0' shape=(50, 10) dtype=float32_ref>}

In [11]:
h_layer = tf.nn.bias_add( tf.matmul( x, weights['hidden'] ), biases['hidden'] )
h_layer = tf.nn.relu(h_layer)
print(h_layer)

"""
out_layer = tf.contrib.layers.fully_connected(inputs=h_layer, num_outputs=output_num_units, activation_fn=tf.nn.relu, biases_initializer=tf.zeros_initializer() )
print(out_layer)

out_layer = tf.layers.batch_normalization(out_layer)

"""
out_layer = tf.nn.bias_add( tf.matmul( h_layer, weights['output']) , biases['output'] )
print(out_layer)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits= out_layer, labels=y))
print(cost)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.initialize_all_variables()

Tensor("Relu:0", shape=(?, 50), dtype=float32)
Tensor("BiasAdd_1:0", shape=(?, 10), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [12]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x / unclean_batch_x.max()
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
  #  print("Batch mask len:" + str(len(batch_mask)))
    #print(batch_mask)
    
    batch_x = train_x[batch_mask]
   # print("Batch_x.shape")
    #print(batch_x.shape)

    batch_x = preproc(batch_x)
    
    if dataset_name == 'train':
        batch_y = train_y[batch_mask]
        batch_y = dense_to_one_hot(batch_y)
     #   print( "Shape of batch_y")
      #  print(batch_y.shape)
        
        
    return batch_x, batch_y

In [13]:
#define session
with tf.Session() as sess:
    sess.run(init)
    
    for ep in range(epochs):
        avg_cost = 0
        total_batch = len(train_x)/batch_size
        #print("total no of batches: " + str(total_batch))
        for bt in range(total_batch):
            batch_x, batch_y = batch_creator( batch_size, len(train_x), 'train')
            _, c = sess.run( [optimizer, cost] ,  feed_dict = { x:batch_x, y: batch_y } )
            avg_cost += c / total_batch
            
            
        print "Epoch:", (ep+1), "cost =", "{:.5f}".format(avg_cost)
    
    print("Training completed")    
    
    pred_temp = tf.equal(tf.argmax(out_layer, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    
    print ( "Validation Accuracy:" )
    print( accuracy.eval({x: val_x, y: dense_to_one_hot(val_y)}) )

    predict = tf.argmax(out_layer, 1)
    print(predict)
    pred = predict.eval({x: test_x})
    
    print("len of pred:" + str(len(pred)))
    
    outFile = open("/home/ace/vertika/test/out.csv", "w")
    outFile.write("ImageId,Label\n")
    for i in range( len(pred)):
        outFile.write( str(i+1) + "," + str(pred[i]) + "\n")
        
    print("File written\n")

Epoch: 1 cost = 4.24516
Epoch: 2 cost = 2.09980
Training completed
Validation Accuracy:
0.7348623
Tensor("ArgMax_2:0", shape=(?,), dtype=int64)
len of pred:28000
File written

